<a href="https://colab.research.google.com/github/NickVoulg02/Information-Retrieval/blob/main/colbert_test_link4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColBERTv2: Indexing & Search Notebook
First, we'll import the relevant classes. Note that `Indexer` and `Searcher` are the key actors here. Next, we'll download the necessary dependencies.

In [1]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2576, done.
remote: Counting objects: 100% (1089/1089), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 2576 (delta 856), reused 828 (delta 779), pack-reused 1487
Receiving objects: 100% (2576/2576), 2.01 MiB | 12.57 MiB/s, done.
Resolving deltas: 100% (1609/1609), done.


In [2]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/30

In [3]:
import colbert

In [4]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

We will use the docs files and the Queries_20 file.

In [47]:
#importing tsv files from personal github repository and creating docs dataset and queries
!git clone https://github.com/NickVoulg02/Information-Retrieval.git
import pandas as pd
from datasets import Dataset
dataset = 'test'
df1 = pd.read_csv("Information-Retrieval/colbert_test/doc_col.tsv", delimiter = '\t', index_col=0)
collection = Dataset.from_pandas(df1, preserve_index=True)
query = Queries("Information-Retrieval/colbert_test/queries_20.tsv")
f'Loaded {len(query)} queries and {len(collection):,} passages'

Cloning into 'Information-Retrieval'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 105 (delta 32), reused 15 (delta 15), pack-reused 59
Receiving objects: 100% (105/105), 563.95 KiB | 4.86 MiB/s, done.
Resolving deltas: 100% (41/41), done.
[Jan 25, 11:47:47] #> Loading the queries from Information-Retrieval/colbert_test/queries_20.tsv ...
[Jan 25, 11:47:47] #> Got 20 queries. All QIDs are unique.



'Loaded 20 queries and 1,209 passages'

In [48]:
print(query[0])
print(collection[0])

WHAT ARE THE EFFECTS OF CALCIUM ON THE PHYSICAL PROPERTIES OF MUCUS FROM CF PATIENTS
{'doc': 'PSEUDOMONAS AERUGINOSA INFECTION IN CYSTIC FIBROSIS OCCURRENCE OF PRECIPITATING ANTIBODIES AGAINST PSEUDOMONAS AERUGINOSA IN RELATION TO THE CONCENTRATION OF SIXTEEN SERUM PROTEINS AND THE CLINICAL AND RADIOGRAPHICAL STATUS OF THE LUNGS THE SIGNIFICANCE OF PSEUDOMONAS AERUGINOSA INFECTION IN THE RESPIRATORY TRACT OF 9 CYSTIC FIBROSIS PATIENTS HAVE BEEN STUDIED BY MEANS OF IMMUNOELECTROPHORETICAL ANALYSIS OF PATIENTS SERA FOR THE NUMBER OF PRECIPITINS AGAINST PSEUDOMONAS AERUGINOSA AND THE CONCENTRATIONS OF 16 SERUM PROTEINS IN ADDITION THE CLINICAL AND RADIOGRAPHICAL STATUS OF THE LUNGS HAVE BEEN EVALUATED USING 2 SCORING SYSTEMS PRECIPITINS AGAINST PSEUDOMONAS AERUGINOSA WERE DEMONSTRATED IN ALL SERA THE MAXIMUM NUMBER IN ONE SERUM WAS 22 THE CONCENTRATIONS OF 12 OF THE SERUM PROTEINS WERE SIGNIFICANTLY CHANGED COMPARED WITH MATCHED CONTROL PERSONS NOTABLY IGG AND IGA WERE ELEVATED AND THE AC

## Indexing

For an efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

In [49]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens

index_name = f'{dataset}.{nbits}bits'

Now run the `Indexer` on the collection subset. Assuming the use of only one GPU, this cell should take about six minutes to finish running.

In [50]:
checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):                   # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)   # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                  # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection["doc"], overwrite=True)




[Jan 25, 11:47:57] #> Note: Output directory /content/experiments/notebook/indexes/test.2bits already exists


[Jan 25, 11:47:57] #> Will delete 10 files already at /content/experiments/notebook/indexes/test.2bits in 20 seconds...
#> Starting...
#> Joined...


In [51]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/content/experiments/notebook/indexes/test.2bits'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

In [52]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection["doc"])

[Jan 25, 11:48:53] #> Loading codec...
[Jan 25, 11:48:53] #> Loading IVF...
[Jan 25, 11:48:53] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 653.83it/s]

[Jan 25, 11:48:53] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 168.24it/s]


In [54]:
question = query[0]

# Find the top-10 passages for this query
results = searcher.search(question, k=20)
#print(results)

passages_ranked = {}
#Print out the top-k retrieved passages
print("Rank\tScore\tId\tPassage")
for passage_id, passage_rank, passage_score in zip(*results):
     print(f"[{passage_rank}] \t{passage_score:.1f} \t{collection['doc_id'][passage_id]} \t{searcher.collection[passage_id]}")
     passages_ranked[passage_id] = passage_rank

Rank	Score	Id	Passage
[1] 	23.0 	533 	EFFECTS OF CALCIUM ON INTESTINAL MUCIN IMPLICATIONS FOR CYSTIC FIBROSIS A MAJOR FEATURE OF THE DISEASE CYSTIC FIBROSIS IS THE EXCESSIVE CONCENTRATION OF MUCUS WITHIN DUCTS AND GLANDS OF MUCOUSPRODUCING ORGANS SOME MUCOUS SECRETIONS ALSO SHOW AN ELEVATION IN CALCIUM CONCENTRATION USING PURIFIED RAT INTESTINAL GOBLET CELL MUCIN AS A MODEL MUCIN WE HAVE INVESTIGATED THE EFFECT OF MILLIMOLAR ADDITIONS 125 MM OF CACL2 ON THE PHYSICAL PROPERTIES OF THE MUCIN ISOTONICITY OF INCUBATION MEDIA WAS PRESERVED IN ORDER TO MIMIC IN VIVO CONDITIONS CACL2 815MM CAUSED A 1533 DECREASE IN VISCOSITY NO CHANGE IN ELECTROPHORETIC MOBILITY IN ACRYLAMIDE GELS AND A 2030 DECREASE IN SOLUBILITY OF THE MUCIN SOLUBILITY CHANGES WERE REVERSED BY THE ADDITION OF EDTA 20 MM TO INCUBATIONS INSOLUBILITY WAS ALSO PRODUCED IN INCUBATIONS OF MUCIN WITH A MIXTURE OF SOLUBLE INTESTINAL CONTENTS NACL WASHINGS THESE FINDINGS STRONGLY SUGGEST THAT THE MUCIN BECAME SMALLER AND MORE DENSE 

Metrics


In [60]:
f = open("Information-Retrieval/Relevant_20", "r")
file = f.read().splitlines()
relevant_docs = []
for line in file:
    line=line.split()
    list = []
    for num in line:
        num = int(num)
        list.append('doc'+str(num))
    relevant_docs.append(list)                  # relevant_docs list includes every line from Relevant_20

Mean Average Precision

In [63]:
avg_pr_list = []
#i=0;
for i in range(20):
    results = searcher.search(query[i], k=400)
    passages_ranked = {}
    for passage_id, passage_rank, passage_score in zip(*results):
      passages_ranked[passage_id] = passage_rank
    precision_at_k = []
    recall_at_k = []
    true_positives = 0

    for key, value in passages_ranked.items():
      doc = "doc"+str(collection['doc_id'][key])
      if(doc in relevant_docs[i]):
        #print(doc)
        true_positives+=1
        #print(value)
        precision_at_k.append(true_positives/value)

    average_precision = 0
    for x in range(len(precision_at_k)):
      average_precision += precision_at_k[x]

    #print(average_precision)
    average_precision = average_precision/len(relevant_docs[i])
    print(average_precision)
    avg_pr_list.append(average_precision)

mean_average_precision = sum(avg_pr_list)/20
print(mean_average_precision)

0.25025136218216215
0.1275958667167708
0.19042972362417085
0.2904289240181963
0.19663148346453152
0.18654405298509594
0.09508615757294149
0.057845607349380226
0.2558358131230472
0.30091203317584414
0.6061873311125703
0.16263126778194076
0.12594239063843188
0.21404384548037259
0.15943431927260845
0.22319432312345341
0.1053961082824507
0.14575299700765537
0.07857902536760272
0.5313826644821248
0.21520526483806762


Mean Reciprocal Rank

In [64]:
mean_rep_rank = 0
for i in range(20):
    results = searcher.search(query[i], k=400)
    passages_ranked = {}
    for passage_id, passage_rank, passage_score in zip(*results):
      passages_ranked[passage_id] = passage_rank

    for key, value in passages_ranked.items():
      doc = "doc"+str(collection['doc_id'][key])
      if(doc in relevant_docs[i]):
            mean_rep_rank += 1/value
            break

mean_rep_rank = mean_rep_rank/20
print(mean_rep_rank)


0.7333333333333333
